## **Sección 1: Manejo de Bases de Datos** 

# **`data_insights_consulting`**

### **Objetivo**
Esta sección tiene como propósito evaluar las habilidades prácticas en el manejo de bases de datos utilizando SQL. Se busca demostrar la capacidad de realizar consultas, emplear funciones de agregación, manejar múltiples tablas y generar vistas que faciliten el análisis de la información.

### **Descripción**
Se cuenta con una base de datos relacional que contiene información de clientes, pedidos, productos y detalles de los pedidos. A partir de estas tablas se desarrollarán las siguientes tareas:

1. Determinar la cantidad total de productos vendidos por cada categoría.
2. Calcular el total acumulado gastado por cada cliente en el tiempo, ordenado por fecha del pedido.
3. Identificar los productos que nunca han sido vendidos.
4. Crear una vista llamada `customer_order_summary` que muestre:  
   - Nombre del cliente  
   - Número total de pedidos  
   - Total gastado  

Los resultados se mostrarán en tablas y se incluirá una interpretación para cada consulta.



### **Carga de datos** 

Se utiliza un archivo `.sqlite` para simular un entorno de base de datos real, permitiendo ejecutar consultas SQL directamente y mantener las relaciones entre tablas sin procesos previos de carga o transformación.


### **Estructura de tablas**

**CUSTOMERS**  
- **id**: Identificador único del cliente.  
- **name**: Nombre completo del cliente.  
- **email**: Correo electrónico del cliente.  
- **created_at**: Fecha de registro del cliente.  

**ORDERS**  
- **id**: Identificador único del pedido.  
- **customer_id**: ID del cliente que realizó el pedido.  
- **order_date**: Fecha en que se realizó el pedido.  
- **total_amount**: Monto total del pedido.  

**ORDER ITEMS**  
- **id**: Identificador único del detalle del pedido.  
- **order_id**: ID del pedido al que pertenece el artículo.  
- **product_id**: ID del producto vendido.  
- **quantity**: Cantidad de unidades vendidas.  
- **unit_price**: Precio unitario del producto.  

**PRODUCTS**  
- **id**: Identificador único del producto.  
- **name**: Nombre del producto.  
- **category**: Categoría del producto.  


In [1]:
import sqlite3
import pandas as pd

# Conexión a la base
conn = sqlite3.connect('../datasets/sqlite/base_datos.sqlite')

# Lectura de las tablas
customers = pd.read_sql_query("SELECT * FROM customers", conn)
orders = pd.read_sql_query("SELECT * FROM orders", conn)
order_items = pd.read_sql_query("SELECT * FROM order_items", conn)
products = pd.read_sql_query("SELECT * FROM products", conn)

In [2]:
customers

,id,name,email,created_at
0,1,Juan Pérez,juan@email.com,2023-01-15
1,2,Laura Gómez,laura@email.com,2023-02-10


In [3]:
orders

,id,customer_id,order_date,total_amount
0,1,1,2023-04-10,150.0
1,2,2,2023-05-20,200.0
2,3,1,2023-06-05,75.0


In [4]:
order_items

,id,order_id,product_id,quantity,unit_price
0,1,1,10,2,50.0
1,2,1,11,1,50.0
2,3,2,12,4,50.0


In [5]:
products

,id,name,category
0,10,Laptop,Electrónica
1,11,Mouse,Electrónica
2,12,Silla Oficina,Muebles


### **1. Productos vendidos por categoría**
Se determina la cantidad total de productos vendidos por cada categoría utilizando las tablas `order_items` y `products`. El objetivo es identificar cuáles categorías han tenido mayor rotación en ventas.


In [6]:
# Se calcula la cantidad total de productos vendidos por categoría
query_cat_sales = """
SELECT 
    p.category AS categoria,
    SUM(oi.quantity) AS total_vendidos
FROM order_items oi
JOIN products p ON oi.product_id = p.id
GROUP BY categoria
ORDER BY total_vendidos DESC;
"""

cat_sales = pd.read_sql_query(query_cat_sales, conn)
cat_sales


,categoria,total_vendidos
0,Muebles,4
1,Electrónica,3


**Conclusión:**  
Se observa que la categoría con mayor número de unidades vendidas es `Muebles`, seguida de `Electrónica`.  
Este resultado sugiere que, dentro del periodo analizado, la demanda se concentra en artículos de mobiliario, lo que podría estar vinculado a campañas comerciales, estacionalidad o cambios en las necesidades del cliente.  
Contar con este tipo de indicadores permite priorizar la gestión de inventarios, optimizar la asignación de recursos y enfocar las estrategias de marketing en las categorías de mayor rotación, sin descuidar aquellas con menor participación para identificar oportunidades de crecimiento.

### **2. Total acumulado gastado por cliente**
Se agrega una columna que muestra el gasto acumulado de cada cliente en el tiempo, ordenado por fecha del pedido. Esto permite identificar patrones de consumo y clientes de alto valor.


In [7]:
# Se calcula el gasto acumulado por cliente ordenado por fecha
query_cumulative = """
SELECT 
    c.name AS cliente,
    o.order_date AS fecha_pedido,
    o.total_amount AS monto_pedido,
    SUM(o.total_amount) OVER (PARTITION BY c.id ORDER BY o.order_date) AS gasto_acumulado
FROM orders o
JOIN customers c ON o.customer_id = c.id
ORDER BY cliente, fecha_pedido;
"""

cumulative = pd.read_sql_query(query_cumulative, conn)
cumulative


,cliente,fecha_pedido,monto_pedido,gasto_acumulado
0,Juan Pérez,2023-04-10,150.0,150.0
1,Juan Pérez,2023-06-05,75.0,225.0
2,Laura Gómez,2023-05-20,200.0,200.0


**Conclusión:**  
Se logra visualizar cómo evoluciona el gasto de cada cliente a lo largo del tiempo, identificando no solo el monto total invertido, sino también la frecuencia y el comportamiento de compra.  

Este indicador es esencial para el diseño de programas de fidelización, ya que permite segmentar clientes según su valor acumulado, anticipar necesidades y priorizar acciones comerciales hacia aquellos con mayor potencial de recompra.  
Además, facilita la detección temprana de clientes inactivos, lo que abre la puerta a estrategias de reactivación específicas.

### **3. Productos nunca vendidos**
Se identifican los productos que no tienen registros en la tabla `order_items`, lo que permite detectar oportunidades para optimizar el catálogo.


In [8]:
# Productos que nunca han sido vendidos
query_unsold = """
SELECT 
    p.id,
    p.name,
    p.category
FROM products p
LEFT JOIN order_items oi ON p.id = oi.product_id
WHERE oi.product_id IS NULL;
"""

unsold = pd.read_sql_query(query_unsold, conn)
unsold


,id,name,category


**Conclusión:**  
La consulta confirma que, en el periodo y dataset analizado, **no existen productos sin ventas registradas**.  
Esto indica que, al menos en esta muestra, la rotación de inventario es completa y no se evidencian referencias inactivas.  

En un contexto real, este resultado serviría como punto de control positivo, aunque se recomienda ejecutar esta validación periódicamente para anticipar posibles acumulaciones de inventario o problemas de comercialización.



### **4. Vista: `customer_order_summary`**
Se crea una vista que consolida información clave por cliente: nombre, número total de pedidos y monto total gastado.


In [9]:
# Creación de la vista
create_view = """
CREATE VIEW customer_order_summary AS
SELECT 
    c.name AS cliente,
    COUNT(o.id) AS total_pedidos,
    SUM(o.total_amount) AS total_gastado
FROM customers c
JOIN orders o ON c.id = o.customer_id
GROUP BY c.name;
"""
conn.execute("DROP VIEW IF EXISTS customer_order_summary")
conn.execute(create_view)

# Consultar la vista
summary = pd.read_sql_query("SELECT * FROM customer_order_summary", conn)
summary


,cliente,total_pedidos,total_gastado
0,Juan Pérez,2,225.0
1,Laura Gómez,1,200.0


**Conclusión:**  
La vista `customer_order_summary` consolida información clave de cada cliente, mostrando el número total de pedidos y el monto total gastado.  
En este caso, se observa que **Juan Pérez** ha realizado 2 pedidos por un valor acumulado de 225, mientras que **Laura Gómez** ha realizado 1 pedido por un valor de 200. 
 
Este tipo de vista es útil para alimentar reportes recurrentes y dashboards comerciales, así como para segmentar clientes según su volumen de compras, priorizar esfuerzos de fidelización y detectar perfiles con mayor potencial de retención o ventas adicionales.

